In [3]:
import pandas as pd

In [4]:
diagnosis_df = pd.read_csv('../data/raw/trainingSet/training_SyncDiagnosis.csv')
patient_df = pd.read_csv('../data/raw/trainingSet/training_SyncPatient.csv')
transcript_df = pd.read_csv('../data/raw/trainingSet/training_SyncTranscript.csv')
conversion_df = pd.read_csv('../data/raw/trainingSet/training_SyncTranscriptDiagnosis.csv')

patient_df.head()

,PatientGuid,DMIndicator,Gender,YearOfBirth,State,PracticeGuid
0,FB6EFC3D-1A20-4497-9CBD-00027CC5D220,0,M,1929,SD,7BF4DAD8-5F67-4985-B911-20C9E89A3737
1,C6746626-6783-4650-A58F-00065649139A,0,F,1985,TX,E7101967-2FF1-4B0F-8129-B0B429D1D15C
2,E05C6E8F-779F-4594-A388-000C635AE4D3,0,F,1984,NJ,FC01A799-1CAF-464F-A86F-8A666AB86F32
3,EAEBD216-F847-4355-87B2-000D942E08F0,0,M,1959,OH,EEBC95EF-79BE-4542-892E-98D3166BAB20
4,C7F10A80-4934-42D2-8540-000FBEBA75C8,0,F,1990,FL,677BA32E-B4C4-48F2-86E4-08C42B135401


In [5]:
transcript_df.VisitYear.value_counts()

2010    48132
2011    43360
2009    19945
2012    15640
0        3954
Name: VisitYear, dtype: int64

In [6]:
def dense_dataframe(df, on):
    """dense the dataframe by combining entries of same value of a given column"""
    lst = []
    for index, value in df.iteritems(): # loop through columns in the dataframe
        # combines rows of values of each column to a list, and store the lists into a list
        lst.append(df.groupby(on)[index].apply(list)) 
        
    processed_df = pd.concat(lst, axis=1) # convert the list of lists of values to a dataframe 
    processed_df[on] = processed_df[on].apply(lambda x:x[0]) # retrieve the same value of 'on' column from the list    
    return processed_df

processed_conversion_df = dense_dataframe(conversion_df, 'TranscriptGuid')
processed_transcripts_df = dense_dataframe(transcript_df, 'PatientGuid')
# pd.merge(processed_diagnosis_df, processed_transcripts_df, 'TranscriptGuid')
# processed_transcript_df = dense_dataframe(transcript_df, 'PatientGuid')
# transcript_diagnosis_df = processed_transcript_df.merge(processed_diagnoses_df, 'TranscriptGuid')
# patient_transcript_df = patient_df.merge(processed_transcript_df, on='PatientGuid')
# patient_transcript_diagnosis_df = patient

In [7]:
conversion_dict1 = processed_conversion_df.set_index('TranscriptGuid')['DiagnosisGuid'].to_dict()
conversion_dict2 = diagnosis_df.set_index('DiagnosisGuid').to_dict()

In [10]:
def get_diagnoses(transcripts):
    diagnoses, descriptions_all, acute_all, icd9_all, start_all, stop_all = [], [], [], [], [], []

    for i in transcripts:
        if i in conversion_dict1:
            diag = conversion_dict1[i]
            diagnoses.append(diag)

            descriptions_one, acute_one, icd9_one, start_one, stop_one = [], [], [], [], []
            for k in diag:
                descriptions_one.append(conversion_dict2['DiagnosisDescription'][k])
                acute_one.append(       conversion_dict2['Acute'][k])
                icd9_one.append(        conversion_dict2['ICD9Code'][k])
                start_one.append(       conversion_dict2['StartYear'][k])
                stop_one.append(        conversion_dict2['StopYear'][k])
            
            descriptions_all.append(descriptions_one)
            acute_all.append(acute_one)
            icd9_all.append(icd9_one)
            start_all.append(start_one)
            stop_all.append(stop_one)
            
    return diagnoses, descriptions_all, acute_all, icd9_all, start_all, stop_all

def combine_data(conversion, transcripts):
    """ Matches diagnosis data with each transcript """
    
    transcripts['CombinedData'] = transcripts.TranscriptGuid.apply(get_diagnoses)
    
    transcripts['DiagnosisGuid']        = [x[0] for x in transcripts['CombinedData']]
    transcripts['DiagnosisDescription'] = [x[1] for x in transcripts['CombinedData']]
    transcripts['Acute']                = [x[2] for x in transcripts['CombinedData']]
    transcripts['ICD9Code']             = [x[3] for x in transcripts['CombinedData']]
    transcripts['StartYear']            = [x[4] for x in transcripts['CombinedData']]
    transcripts['StopYear']             = [x[5] for x in transcripts['CombinedData']]
        
    return transcripts
        
combined_data = combine_data(processed_conversion_df, processed_transcripts_df)
combined_data.to_csv('combined_data.csv')